In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
import wandb
import os
import random
from time import time

from lib.data import EpisodeDataset, EpisodeDataset_Unmodified
from lib.neural_networks import FullyConnected_Continuous, FullyConnected_Continuous_2

In [47]:
WIDTH = 5
OBS_SIZE = 30 #46 is standard, 15 is minimal
HIDDEN_SIZE = 2048

INCLUDE_CHANGE = True
CORRUPT = True

In [48]:
TRAIN_PATH_1 = '/data/hmead/Episode Data/Continuous Random Data/30 TPS/3v3 Train'

train_dataset = EpisodeDataset_Unmodified([TRAIN_PATH_1], width=WIDTH, include_change=INCLUDE_CHANGE, corrupt=CORRUPT)

print(len(train_dataset))

Loading Episode Data
Finished Loading Episode Data
125820


In [49]:
Train_Data_Loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=32, pin_memory=True)

In [50]:
TEST_PATH_UNFILTERED = '/data/hmead/Episode Data/Continuous Random Data Unfiltered/30 TPS/3v3 Test'
TEST_PATH = '/data/hmead/Episode Data/Continuous Random Data/30 TPS/3v3 Test'
TEST_PATH_HUMAN = '/data/hmead/Episode Data/Human Example Data/30 TPS'


test_dataset_unfiltered = EpisodeDataset_Unmodified([TEST_PATH_UNFILTERED], width=WIDTH, include_change=INCLUDE_CHANGE, corrupt=CORRUPT)
test_dataset = EpisodeDataset_Unmodified([TEST_PATH], width=WIDTH, include_change=INCLUDE_CHANGE, corrupt=CORRUPT)
test_dataset_human = EpisodeDataset_Unmodified([TEST_PATH_HUMAN], width=WIDTH, include_change=INCLUDE_CHANGE, corrupt=CORRUPT)


print(len(test_dataset_unfiltered))
print(len(test_dataset))
print(len(test_dataset_human))

Loading Episode Data
Finished Loading Episode Data
Loading Episode Data
Finished Loading Episode Data
Loading Episode Data
Finished Loading Episode Data
180
180
129


In [51]:
Test_Data_Loader_Unfiltered = DataLoader(test_dataset_unfiltered, batch_size=1, shuffle=False, num_workers=1)
Test_Data_Loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=1)
Test_Data_Loader_Human = DataLoader(test_dataset_human, batch_size=1, shuffle=False, num_workers=1)

In [52]:
#model = ConvNet(obs_size=OBS_SIZE, obs_width=WIDTH, conv_number=10, hidden_size=HIDDEN_SIZE)
model = FullyConnected_Continuous_2(obs_size=OBS_SIZE, obs_width=WIDTH, hidden_size=HIDDEN_SIZE)

gpumodel = model.cuda()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(gpumodel.parameters(), lr=0.0001)
#scheduler = ExponentialLR(optimizer, gamma = 0.63)

In [11]:
gpumodel.load_state_dict(torch.load('trained_networks/mild-wood-119.pt'))

<All keys matched successfully>

In [53]:
def test_model(gpumodel, Test_Data_Loader):
    gpumodel.eval()

    total_loss = 0
    obs_count = 0

    for _obs, _act, _add_data in Test_Data_Loader:
        # EVALUATION
        obs = _obs[0].cuda()
        act = _act[0, :, 0:5].float().cuda()

        if obs.shape[0] == 0:
            continue

        with torch.no_grad():
            y_pred = model(obs)

        loss = criterion(y_pred, act)

        # LOGGING
        obs_count += len(obs)
        total_loss += loss.item() * len(obs)

    return total_loss / obs_count

In [54]:
wandb.init(project="Inverse Dynamics Model Retest", entity="harrymead")

wandb.config = {
  "learning_rate": 0.0001,
  "epochs": 10,
  "batch_size": 10
}

loss_MSE,█▆▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss_MSE,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss_human_MSE,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂▁▁▁▂▁▁▂▂▁▁▁▁▂▁
test_loss_unfiltered_MSE,█▅▄▄▅▃▃▁▂▂▁▁▂▂▁▂▁▁▁▂▂▂▂▂▁▂▁▂▂▂▂▂▂▂▂▁▃▂▂▃
loss_MSE,0.04743
test_loss_MSE,0.05162
test_loss_human_MSE,0.08279
test_loss_unfiltered_MSE,0.30888


In [55]:
BUFFER_SIZE = 5

obs_buffer = []
act_buffer = []

gpumodel.train()

for epoch in range(30):
    batch = 0
    total_loss = 0
    obs_count = 0

    for _obs, _act, _add_data in Train_Data_Loader:
        # OBS BATCHING 
        obs = _obs[0].cuda()
        act = _act[0, :, 0:5].float().cuda()

        if obs.shape[0] == 0:
            continue

        obs_buffer.append(obs)
        act_buffer.append(act)

        if len(obs_buffer) < BUFFER_SIZE:
            continue

        obs = torch.cat(obs_buffer)
        act = torch.cat(act_buffer)

        obs_buffer = []
        act_buffer = []

        # TRAINING

        optimizer.zero_grad()

        y_pred = model(obs)
        
        loss = criterion(y_pred, act)

        loss.backward()
        optimizer.step()


        # LOGGING
        obs_count += len(obs)
        total_loss += loss.item() * len(obs)


        batch += 1
        if batch % 1000 == 0:
            if (batch - 1000) % 5000 == 0:
                start_test = time()
                test_loss_unfiltered = test_model(gpumodel, Test_Data_Loader_Unfiltered)
                test_loss = test_model(gpumodel, Test_Data_Loader)
                test_loss_human = test_model(gpumodel, Test_Data_Loader_Human)
                gpumodel.train()

            print(f'Epoch {epoch + 1} - Loss: {total_loss / obs_count}, Test Loss Unfiltered: {test_loss_unfiltered}, Test Loss: {test_loss}')
            wandb.log({"loss_MSE": total_loss/obs_count,
                        "test_loss_unfiltered_MSE": test_loss_unfiltered,
                        "test_loss_MSE": test_loss,
                        "test_loss_human_MSE": test_loss_human})

            total_loss = 0
            obs_count = 0
    

    #scheduler.step()
    if (epoch % 2) == 0:
        torch.save(gpumodel.state_dict(), f"trained_networks/{wandb.run.name}/{wandb.run.name} ({epoch} epochs).pt")

Epoch 1 - Loss: 0.1515716114217394, Test Loss Unfiltered: 0.32164698179778417, Test Loss: 0.11128946760630808
Epoch 1 - Loss: 0.1023655923355818, Test Loss Unfiltered: 0.32164698179778417, Test Loss: 0.11128946760630808
Epoch 1 - Loss: 0.087814518914897, Test Loss Unfiltered: 0.32164698179778417, Test Loss: 0.11128946760630808
Epoch 1 - Loss: 0.08119788926482034, Test Loss Unfiltered: 0.32164698179778417, Test Loss: 0.11128946760630808
Epoch 1 - Loss: 0.07567844624587301, Test Loss Unfiltered: 0.32164698179778417, Test Loss: 0.11128946760630808
Epoch 1 - Loss: 0.07381480685570897, Test Loss Unfiltered: 0.3046861676444733, Test Loss: 0.06839173380105829
Epoch 1 - Loss: 0.07111509199248346, Test Loss Unfiltered: 0.3046861676444733, Test Loss: 0.06839173380105829
Epoch 1 - Loss: 0.06849870844238812, Test Loss Unfiltered: 0.3046861676444733, Test Loss: 0.06839173380105829
Epoch 1 - Loss: 0.0668548161974949, Test Loss Unfiltered: 0.3046861676444733, Test Loss: 0.06839173380105829
Epoch 1 - 

In [45]:
test_model(gpumodel, Test_Data_Loader_Human)

0.05441522278061852

In [12]:
conf_mtx = torch.zeros((8,3,3)).cuda()
CONF_MAT = True

gpumodel.eval()

total_loss = 0
obs_count = 0
total_correct = torch.zeros(9).cuda()

correct_on_ground = torch.zeros(9).cuda()
total_on_ground = 0

for _obs, _act, _add_data in Test_Data_Loader:
    # EVALUATION
    obs = _obs[0].cuda()
    act = torch.cat((_act[0], _add_data[0][:,0:1]), dim=1).long().cuda()
    add_data = _add_data[0].long()

    if obs.shape[0] == 0:
        continue

    with torch.no_grad():
        y_pred = model(obs)

    y_pred = model(obs)
    pred_act = torch.zeros(obs.shape[0], 9).cuda()

    loss = 0
    for i in range(9):
        loss += criterion(y_pred[i], act[:, i])

        pred_act[:, i] = torch.argmax(y_pred[i], dim=1)


    # LOGGING
    obs_count += len(obs)
    total_loss += loss.item() * len(obs)

    correct = act == pred_act
    total_correct += torch.sum(correct, dim=0)

    on_ground = pred_act[:,8]

    #print(correct)
    #print(on_ground.bool())

    total_on_ground += torch.sum(on_ground == 1).item()


    correct_on_ground += torch.sum(correct[on_ground == 1], dim=0)

    if CONF_MAT:
        for i in range(obs.shape[0]):
            for j in range(8):
                if on_ground[i] == 1:
                    conf_mtx[j, act[i, j], pred_act[i, j].long()] += 1

    

print(f'Loss: {total_loss / obs_count}, Correct: {total_correct / obs_count}, Correct On Ground: {correct_on_ground / total_on_ground}, On Ground: {total_on_ground / obs_count}')

np.set_printoptions(precision=4, suppress=True)
print((conf_mtx).cpu().numpy())

Loss: 26.654266357421875, Correct: tensor([0.9120, 0.8223, 0.8248, 0.4793, 0.8872, 0.9554, 0.9545, 0.9802, 0.2843],
       device='cuda:0'), Correct On Ground: tensor([0.9169, 0.8199, 0.9977, 0.4163, 1.0000, 0.9867, 0.9555, 0.9723, 0.0000],
       device='cuda:0'), On Ground: 0.715702479338843
[[[ 183.    6.    6.]
  [  30.  359.   33.]
  [   5.   64. 1046.]]

 [[ 297.   45.    3.]
  [  67.  608.  118.]
  [  10.   69.  515.]]

 [[   0.    0.    0.]
  [   1. 1728.    3.]
  [   0.    0.    0.]]

 [[   0.    0.    0.]
  [ 368.  721.  643.]
  [   0.    0.    0.]]

 [[   0.    0.    0.]
  [   0. 1732.    0.]
  [   0.    0.    0.]]

 [[1709.   23.    0.]
  [   0.    0.    0.]
  [   0.    0.    0.]]

 [[1380.   50.    0.]
  [  27.  275.    0.]
  [   0.    0.    0.]]

 [[1684.   48.    0.]
  [   0.    0.    0.]
  [   0.    0.    0.]]]
